# Attributes Exploration

This notebook explores the different attributes variables provided for different businesses in the Yelp business.json file. These attributes variables provide more colorful information, such as costliness, forms of accepted payment (e.g., credit card or bitcoin), and parking options, on the businesses in the Yelp dataset. The attributes are stored in a sub-dictionary within each business instance dictionary, and there are further sub-dictionaries within the attributes section. 

Attributes are not consistently reported for each business. Some businesses in the business.json file have many of the possible attributes variables recorded, while some have a few, one, or none.

In [1]:
import json
import yaml
import os
import pandas as pd
import psycopg2

/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Example of attributes for a business

The cell below provides an example attributes section for one business in the business.json file. Attributes itself is a sub-dictionary within the business dictionary. Some of the attributes variables, like RestaurantsReservations or HasTV, are simple key, value pairs. For these cases, the attributes variables are referred to as "Main Keys."

Other attributes variables, like GoodForMeal and BusinessParking are themselves sub-dictionaries within the attributes sub-dictionary and consist of multiple sub-key, sub-value pairs. For these cases, the attributes variables are referred to as "Sub-Keys."

In [ ]:
"attributes":{"RestaurantsReservations":"True",
              "GoodForMeal":"{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}",
              "BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
              "Caters":"True","NoiseLevel":"u'loud'","RestaurantsTableService":"True","RestaurantsTakeOut":"True",
              "RestaurantsPriceRange2":"2","OutdoorSeating":"False","BikeParking":"False",
              "Ambience":"{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}",
              "HasTV":"False","WiFi":"u'no'","GoodForKids":"True","Alcohol":"u'full_bar'","RestaurantsAttire":"u'casual'",
              "RestaurantsGoodForGroups":"True","RestaurantsDelivery":"False"}

### What is the exhaustive list of attribute variables?

Because the attributes variables are recorded inconsistently, as stated above, in the business.json file, the first step in attributes exploration is to determine the exhaustive list of attributes variables recorded in the file. The code below pulls out all possible attributes variables in the business.json file and further denotes when certain attribute variables are represented as simple key, value pairs, or when they are themselves broken into sub-variables.

In [ ]:
#Before running code to explore the attributes variables, first specify the path as a string
# to the business.json file on your local machine. Do not include the file name 'business.json'
#Example: '/Users/firstname.lastname/Desktop'
dataset_path = ''

In [ ]:
data = []

#Create a list titled data that contains the attributes sub-dictionary for each business
with open(dataset_path + os.sep + 'business.json', 'r', encoding = 'utf8') as f:
    for row in f:
        row_dict = json.loads(row)
        data.append(row_dict['attributes'])

main_keys = set()

#Iterate through every business to add attributes main keys into a set, which will contain 
#one of each main key. A set was ideal; it does not record duplicate values
for i in range(0,len(data)):
    if data[i] is not None:
        for k in data[i].keys():
            main_keys.add(k)

#Prints out main keys
print('Main Keys:\n{}\n'.format(main_keys))

#Iterate through the set of main keys looking for all possible sub-keys among the businesses
for x in main_keys:
    x_keys = set()
    for i in range(0,len(data)):
        if data[i] is not None:
            #This makes sure the main key  is actually recorded in the Attributes section of the business in question
            if x in data[i].keys():
                #This checks to see if the main key has sub-keys, which would mean that the main key's value is a dictionary
                if type(yaml.load(data[i][x])) is dict:
                    sub_dict = yaml.load(data[i][x])
                    for k in sub_dict.keys():
                        x_keys.add(k)

    if len(x_keys) > 0:
        print('{} Sub-keys:\n{}\n'.format(x,x_keys))
    else:
        print('{} has no Sub-keys\n'.format(x))

### The attributes variables

The output from the code above should align with the results printed out after executing the code below. In total, there are 39 Main Keys with 7 of these Main Keys containing several Sub-keys. 


In [ ]:
keys = [['BYOB', 'None'], ['RestaurantsCounterService', 'None'], ['GoodForMeal', 'dessert, latenight, lunch, dinner, brunch, breakfast'], ['AcceptsInsurance', 'None'], ['RestaurantsDelivery', 'None'], ['Smoking', 'None'], ['DietaryRestrictions', 'dairy-free, gluten-free, vegan, kosher, halal, soy-free, vegetarian'], ['RestaurantsPriceRange2', 'None'], ['BusinessParking', 'garage, street, validated, lot, valet'], ['OutdoorSeating', 'None'], ['GoodForKids', 'None'], ['WiFi', 'None'], ['CoatCheck', 'None'], ['BusinessAcceptsBitcoin', 'None'], ['Alcohol', 'None'], ['BestNights', 'monday, tuesday, wednesday, thursday, friday, saturday, sunday'], ['DriveThru', 'None'], ['GoodForDancing', 'None'], ['BusinessAcceptsCreditCards'], ['RestaurantsGoodForGroups', 'None'], ['RestaurantsTableService', 'None'], ['ByAppointmentOnly', 'None'], ['HasTV', 'None'], ['Corkage', 'None'], ['Caters', 'None'], ['Ambience', 'touristy, hipster, romantic, divey, intimate, trendy, upscale, classy, casual'], ['RestaurantsReservations', 'None'], ['RestaurantsAttire', 'None'], ['Open24Hours', 'None'], ['Music', 'dj, background_music, jukebox, live, video, karaoke, no_music'], ['DogsAllowed', 'None'], ['AgesAllowed', 'None'], ['NoiseLevel', 'None'], ['WheelchairAccessible', 'None'], ['RestaurantsTakeOut', 'None'], ['BYOBCorkage', 'None'], ['BikeParking', 'None'], ['HappyHour', 'None'], ['HairSpecializesIn', 'straightperms, coloring, extensions, africanamerican, curly, kids, perms, asian']]
df1 = pd.DataFrame(keys, columns = ['Main Key', 'Sub-Keys'])
blankIndex=[''] * len(df1)
df1.index=blankIndex
pd.set_option('display.max_colwidth', -1)
df1

### Raw attributes variables counts

The next step is to count the number of times each attributes Main Key appears in the dataset. Or, in other words, to count the number of businesses for each variable that have a value for the variable-in-question recorded. The motivation for doing so is to see whether some variables appear infrequently enough to warrant their removal from consideration/analysis.

Specifically, the code below counts the number of times each Main Key appears in the dataset without providing specific counts for the Sub-Keys. The reasoning is that each time a Sub-Key appears, by default, its Main Key also appears.

In [ ]:
main_key_count = []

for x in main_keys:
    k = 0
    for i in range(0,len(data)):
        if data[i] is not None and x in data[i].keys():
            k += 1
    main_key_count.append([x,k])
    print('The {} Main Key appears {} times in the data\n'.format(x,k))

In [ ]:
df2 = pd.DataFrame(main_key_count, columns = ['Main Key', 'Count'])
df3 = pd.merge(df1, df2, on="Main Key")
pd.set_option('display.max_colwidth', -1)
df3

The code and output below highlights cells whose count is less than 5000. These attributes variables were deemed too "sparse" for analysis and eventual inclusion in models, so they were removed from consideration.

In [ ]:
df3.style.apply(lambda x: ["background-color: #ff33aa" 
                          if (i >= 2 and v < 5000) 
                          else "" for i, v in enumerate(x)], axis = 1)

Removing the attributes variables above who are recorded less than 5000 times in the dataset leaves 26 Main Key attributes variables. However, this does not reflect the actual number of variables under consideration, as several of these Main Key variables consist of sub-keys, or sub-variables. Removing the Main Key attribute variables with sub-variables and replacing them with their sub-variables leaves 49 attributes variables for consideration. These 49 variables were ultimately ingested into our database.

### Making more sense of the attributes variables counts...

While it is useful to know the raw counts for each of the attributes variables under consideration, it would be more informative to know each variable's count within the businesses that are represented in the review.json file in the Yelp dataset, as the Yelp reviews stored in this file are at the crux of our analysis. It seems plausible that many of the attributes variables would be relatively more represented among review data.

To better understand this idea, let's take a look at some of the data in the review and business tables.

In [2]:
#First establish connection to PostGreSQL database with Yelp data
dbname = ''
username = ''
host = ''
password = ''

conn = psycopg2.connect('dbname={} user={} host={} password={}'.format(dbname, username, host, password))
cur = conn.cursor()

In [ ]:
cur.execute("""select column_name from information_schema.columns where table_name='review'
""")
result = cur.fetchall()
review_columns = [result[i][0] for i in range(0, len(result))]
cur.execute("""select * from review LIMIT 10
""")
review_sample = pd.DataFrame(cur.fetchall(), columns=review_columns)
blankIndex=[''] * len(review_sample)
review_sample.index=blankIndex
review_sample

As shown above, the review table contains a field indicating the business_id of the business associated with the review. These unique business_ids can be summed to produce counts of the number of reviews per business recorded in the Yelp dataset. It is crucial to note that the business.json file includes a field called 'review_count,' but this does not correspond with the nnumber of times a review for a certain business was recorded in the Yelp dataset. As shown below, there are discrepancies between the total number of times a business has ever been reviewed (review_count or total_reviews), as of the collection date of the Yelp data, and the number of times a review for a business was recorded in the dataset. The differences do not appear too large, but the distinction is important.

In [ ]:
cur.execute("""select review.business_id, count(distinct(review_id)) as number_of_reviews, review_count as total_reviews
from review join business on review.business_id = business.business_id
group by review.business_id, review_count limit 30
""")

sample_review_counts = pd.DataFrame(cur.fetchall(), columns = ['business_id', 'number_of_reviews', 'total_reviews'])
blankIndex=[''] * len(sample_review_counts)
sample_review_counts.index=blankIndex
sample_review_counts

### Calculating attributes variables counts based on representation in review data

As alluded to above, it is only so useful to understand how many times the attributes variables appear in the business data, or in other words, the number of businesses with recorded values for the attributes variables. What is more important to know is how many times each attributes variable appears in association with a reviewed business in the review data. If business ABC has values recorded for the DogsAllowed and RestaurantsCounterService attributes variables and has 23 reviews in the review data, then these two attributes variables would add 23 to their counts.

Below is code to calculate the number of times each attributes variable is recorded in connection with a reviewed business in the review data.

In [3]:
#Extract all attributes variables (columns) and append to a list
cur.execute("""select column_name from information_schema.columns where table_name='attributes'
""")
result = cur.fetchall()
attributes_columns = [result[i][0] for i in range(1, 50)]


In [4]:
attributes_counts_reviews = []
for x in attributes_columns:
    cur.execute("""select count({}), count(distinct(review.business_id)) from review join attributes on attributes.business_id = review.business_id where {} is not null""".format(x,x))
    result = cur.fetchall()
    attributes_counts_reviews.append([x,result[0][0],result[0][1]])

In [10]:
#Add highlighting for cells < 50%
counts = pd.DataFrame(attributes_counts_reviews, columns = ['Attribute', 'Attr. Count', 'Distinct Business Count'])
counts['Attr. Count/Reviews'] = counts['Attr. Count']/6685900
counts['Attr. Count/Reviews'] = pd.Series(["{0:.2f}%".format(val * 100) for val in counts['Attr. Count/Reviews']])
counts['Distinct Business Count/Businesses'] = counts['Distinct Business Count']/192609
counts['Distinct Business Count/Businesses'] = pd.Series(["{0:.2f}%".format(val * 100) for val in counts['Distinct Business Count/Businesses']])
counts

,Attribute,Attr. Count,Distinct Business Count,Attr. Count/Reviews,Distinct Business Count/Businesses
0,restaurants_price_range2,5507717,108079,82.38%,56.11%
1,noise_level,4066496,43806,60.82%,22.74%
2,restaurants_attire,4047327,48591,60.54%,25.23%
3,alcohol,4230554,48383,63.28%,25.12%
4,dj,774504,4992,11.58%,2.59%
5,background_music,695302,4478,10.40%,2.32%
6,jukebox,697820,4507,10.44%,2.34%
7,live,701517,4522,10.49%,2.35%
8,video,695467,4483,10.40%,2.33%
9,karaoke,695835,4481,10.41%,2.33%
